<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# HubSpot - Send sales brief
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/HubSpot/HubSpot_Send_sales_brief.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Send+sales+brief:+Error+short+description">Bug report</a>

**Tags:** #hubspot #crm #sales #deal #naas_drivers #notification #asset #emailbuilder #scheduler #naas #analytics #automation #email #text #plotly #html #image

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-04-12 (Created: 2022-02-21)

**Description:** This notebook send a sales brief based on your HubSpot activity.

## Input

### Import libraries

In [ ]:
import os
import json
import naas 
import naas_drivers
from naas_drivers import emailbuilder, hubspot
import pandas as pd
from datetime import datetime
try:
    import tiktoken
except:
    !pip install tiktoken --user
    import tiktoken

### Setup HubSpot
👉 Starting November 30, 2022, HubSpot API keys no longer enable access to HubSpot APIs, so in Naas version 2.8.3 and above, you need [create a private app and use the access token](https://developers.hubspot.com/docs/api/private-apps).

In [ ]:
# Enter Your Access Token
HS_ACCESS_TOKEN = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"

### Get pipelines data

In [ ]:
df_pipelines = hubspot.connect(HS_ACCESS_TOKEN).pipelines.get_all()
df_pipelines

### Setup variables

In [ ]:
# Define scenario 
is_demo = True # Default to True
demo_file_path = "/home/ftp/abi/inputs/demo_data-hubspot_deals.csv"

# For HubSpot 
#pipeline_id = "8432671" #Pick the pipeline_id from df_pipelines output
objective = 50000 #Set the objective of revenue associate to the pipeline_id
#currenry = "$" #not yet implemented

# For Emails
email_to = ["jeremy@naas.ai"]
email_subject = (f"⚡️ Sales Engine Demo - Email Update, {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
DATE_FORMAT = "%Y-%m-%d"


# For AI Chat Plugin
plugin_name = "⚡️ Sales Agent Demo"
plugin_model = "gpt-3.5-turbo-16k"
plugin_temperature = 0
plugin_max_tokens = 8192
system_prompt_max_tokens = 2084

# For Asset Generation
output_dir = "/home/ftp/abi//outputs/"
csv_file_name = "../outputs/df_sales.csv"
image_file_name = "../outputs/image.png"
plugin_file_name = "../outputs/plugin.json"

## Model

### Setup directories

In [ ]:
# Check if directory exists and create it if not
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# Generate outputs files path
csv_file_path = os.path.join(output_dir, csv_file_name)
image_file_path = os.path.join(output_dir, image_file_name)
plugin_file_path = os.path.join(output_dir, plugin_file_name)
print('📂 CSV file path:', csv_file_path)
print('📂 Image file path:', image_file_path)
print('📂 Plugin file path:', plugin_file_path)

### Query Deals

In [ ]:
def get_or_load_data(
    is_demo
    #demo_file_path
    #csv_file_path
):
    
    #Init
    df = pd.DataFrame()
    
    # Check if demo_mode is True
    if is_demo:   
        file_path = "../inputs/demo_data-hubspot_deals.csv"
        if os.path.exists(file_path):
            # Read file
            df = pd.read_csv(file_path)
            df.to_csv(csv_file_path, index=False)
        else: 
            print("Files does not exist.")

    else:
        # If the file doesn't exist, read output
        file_path = "../outputs/df_sales.csv"
        if not os.path.exists(file_path):
            %run  "../models/HubSpot_Create_sales_dataset.ipynb"
        # Read file
        df = pd.read_csv(file_path)
    return df
    
df_sales_c = get_or_load_data(is_demo)
df_sales_c

### Create sales pipeline agregated by dealstages

In [ ]:
df_details = df_sales_c.copy()

# Groupby
to_group = ["dealstage_label", "probability", "displayOrder"]
to_agg = {"amount": "sum", "dealname": "count", "forecasted": "sum"}
df_details = df_details.groupby(to_group, as_index=False).agg(to_agg)

# Sort
df_details = df_details.sort_values("displayOrder")

df_details

### Formatting functions

In [ ]:
def format_number(num):
    NUMBER_FORMAT = "{:,.0f} €"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

In [ ]:
def format_pourcentage(num):
    NUMBER_FORMAT = "{:,.0%}"
    num = str(NUMBER_FORMAT.format(num))
    return num

In [ ]:
def format_varv(num):
    NUMBER_FORMAT = "+{:,.0f} €"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

### Calculate email parameters

In [ ]:
forecasted = df_details.forecasted.sum()
forecasted

In [ ]:
won = df_details[df_details["probability"] == 1].forecasted.sum()
won

In [ ]:
weighted = df_details[df_details["probability"] < 1].forecasted.sum()
weighted

In [ ]:
completion_p = forecasted / objective
completion_p

In [ ]:
completion_v = objective - forecasted
completion_v

In [ ]:
today = datetime.now().strftime(DATE_FORMAT)
today

### Get pipeline details

In [ ]:
df = df_details.copy()

details = []

for _, row in df.iterrows():
    # status part
    dealstage = row.dealstage_label
    probability = row.probability
    detail = f"{dealstage} ({format_pourcentage(probability)})"

    # amount part
    amount = row.amount
    number = row.dealname
    forecasted_ = row.forecasted
    if probability < 1 and probability > 0:
        detail = f"{detail}: <ul><li>Amount : {format_number(amount)}</li><li>Number : {number}</li><li>Weighted amount : <b>{format_number(forecasted_)}</b></li></ul>"
    else:
        detail = f"{detail}: {format_number(amount)}"

    details += [detail]

details

### Get inactives deals

In [ ]:
df_inactive = df_sales_c.copy()

df_inactive.hs_lastmodifieddate = pd.to_datetime(
    df_inactive.hs_lastmodifieddate
).dt.strftime(DATE_FORMAT)

df_inactive["inactive_time"] = (
    datetime.now() - pd.to_datetime(df_inactive.hs_lastmodifieddate, format=DATE_FORMAT)
).dt.days
df_inactive.loc[(df_inactive["inactive_time"] > 30, "inactive")] = "inactive"
df_inactive = (
    df_inactive[
        (df_inactive.inactive == "inactive")
        & (df_inactive.amount != 0)
        & (df_inactive.probability > 0.0)
        & (df_inactive.probability < 1)
    ]
    .sort_values("amount", ascending=False)
    .reset_index(drop=True)
)

df_inactive

In [ ]:
inactives = []

for _, row in df_inactive[:10].iterrows():
    # status part
    dealname = row.dealname
    dealstage_label = row.dealstage_label
    amount = row.amount
    probability = row.probability
    inactive = f"{dealname} ({dealstage_label}): <b>{format_number(amount)}</b>"
    inactives += [inactive]

inactives

### Create pipeline waterfall

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    go.Waterfall(
        name="20",
        orientation="v",
        measure=["relative", "relative", "total", "relative", "total"],
        x=["Won", "Pipeline", "Forecast", "Missing", "Objective"],
        textposition="outside",
        text=[
            format_number(won),
            format_varv(weighted),
            format_number(forecasted),
            format_varv(completion_v),
            format_number(objective),
        ],
        y=[won, weighted, forecasted, completion_v, objective],
        decreasing={"marker": {"color": "#33475b"}},
        increasing={"marker": {"color": "#33475b"}},
        totals={"marker": {"color": "#ff7a59"}},
    )
)


fig.update_layout(title="Sales Metrics", plot_bgcolor="#ffffff", hovermode="x")
fig.update_yaxes(tickprefix="€", gridcolor="#eaeaea")
fig.show()

In [ ]:
fig.write_image(image_file_name)

params = {"inline": True}

graph_image = naas.asset.add(image_file_name)

### Create Naas Chat plugin

In [ ]:
system_prompt = f"""Act as a Sales Agent who has access to the Deals data from the CRM. 
Your role is to manage and nurture these deals, ensuring that they progress through the sales pipeline effectively. 
You will need to communicate with potential clients, understand their needs, and propose solutions that align with their goals. 
Utilize the information in the CRM to personalize your approach, track interactions, and manage follow-ups. 
Your ultimate goal is to close deals successfully, contributing to the overall sales targets of the company."
The fist message should be about presenting yourself with maximum 5 bullet points and displaying the current pipeline status to be displayed as png inside the markdown of the chat: {graph_image}
Then, wait for the first answer from the user, and then start with a first high level analysis.
Here data from the CRM you should focus on: {df_inactive}
"""
system_prompt

### Check token count 

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

system_prompt_tokens = num_tokens_from_string(system_prompt, "cl100k_base")
if system_prompt_tokens > system_prompt_max_tokens:
    print("⚠️ Be carefull, your system prompt looks too big. Tokens:", system_prompt_tokens)
else:
    print("✅ System prompt tokens count OK:", system_prompt_tokens)

### Generate Plugin

In [ ]:
# Create json
plugin = {
    "name": plugin_name,
    "model": plugin_model,
    "temperature": plugin_temperature,
    "max_tokens": plugin_max_tokens,
    "prompt": system_prompt,
}

# Save dict to JSON file
with open(plugin_file_name, "w") as f:
    json.dump(plugin, f)
print("💾 Plugin successfully saved:")

plugin = naas.asset.add(plugin_file_name, params={"inline": True})

### Create email

In [ ]:
def email_brief(
    today,
    forecasted,
    won,
    weighted,
    objective,
    completion_p,
    completion_v,
    details,
    inactives,
):
    content = {
        #'element': naas_drivers.emailbuilder.title("⚡️ Sales Engine - Email Update"),
        #'heading': naas_drivers.emailbuilder.heading(f"Date:{today}"),
        'title': ("⚡️ Sales Engine - Email Update"),
        'heading': (f"Date:{today}"),
        "txt_intro": (
            f"Hi there,<br><br>" f"Here is your daily sales email as of {today}."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="27px", text_align="center", bold=True
        ),
        "text_1": emailbuilder.text(
            f"As of today, your yearly forecasted revenue is {format_number(forecasted)}."
        ),
        "list_1": emailbuilder.list(
            [
                f"Won : {format_number(won)}",
                f"Weighted pipeline : <b>{format_number(weighted)}</b>",
            ]
        ),
        "text_1_2": emailbuilder.text(
            f"You need to find 👉 <u>{format_number(completion_v)}</u> to reach your goal !"
        ),
        "text_1_1": emailbuilder.text(
            f"Your yearly objective is {format_number(objective)} ({format_pourcentage(completion_p)} completion)."
        ),
        "image_1": emailbuilder.image(graph_image, link=graph_image),
        "title_2": emailbuilder.text(
            "🚀 Pipeline", font_size="27px", text_align="center", bold=True
        ),
        "list_2": emailbuilder.list(details),
        "title_3": emailbuilder.text(
            "🧐 Actions needed", font_size="27px", text_align="center", bold=True
        ),
        "text_3": emailbuilder.text("Here are deals where you need to take actions :"),
        "list_3": emailbuilder.list(inactives),
        "text_3_1": emailbuilder.text(
            "If you need more details, connect to Hubspot with the link below."
        ),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin}"),
            text="Start Chatting With Agent",
            background_color="#181a1c",
        ),
        "title_4": emailbuilder.text(
            "Glossary", text_align="center", bold=True, underline=True
        ),
        "list_4": emailbuilder.list(
            [
                "Yearly forecasted revenue :  Weighted amount + WON exclude LOST",
                "Yearly objective : Input in script",
                "Inactive deal : No activity for more than 30 days",
            ]
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    today,
    forecasted,
    won,
    weighted,
    objective,
    completion_p,
    completion_v,
    details,
    inactives,
)

## Output

### Send email

In [ ]:
naas.notification.send(email_to, email_subject, email_content)